# Azure Maps Traffic Incidents

https://azure.microsoft.com/en-us/products/azure-maps/?msockid=2e39c66c693c66a5151fd200687567d0

https://learn.microsoft.com/en-us/azure/azure-maps/

In [1]:
import os
import json
import folium
import pandas as pd
import requests
import sys

from dotenv import load_dotenv
from IPython.display import IFrame, FileLink

In [2]:
sys.version

'3.10.14 (main, May  6 2024, 19:42:50) [GCC 11.2.0]'

In [3]:
load_dotenv("azure.env")

subscription_key = os.getenv('AZURE_MAPS_KEY')

In [4]:
RESULTS_DIR = "results"

os.makedirs(RESULTS_DIR, exist_ok=True)

## Helper

In [5]:
def get_traffic_incidents(bbox, subscription_key=None, bearer_token=None):
    """
    Fetch traffic incidents from Azure Maps.
    
    Parameters:
        bbox (str): Bounding box in format "lon1,lat1,lon2,lat2".
        api_key (str, optional): Azure Maps subscription key.
        bearer_token (str, optional): Azure AD bearer token.
    
    Returns:
        dict: JSON response with traffic incidents (if successful).
    """
    url = "https://atlas.microsoft.com/traffic/incident"
    params = {"api-version": "2025-01-01", "bbox": bbox}

    headers = {}
    if subscription_key:
        headers["Subscription-Key"] = subscription_key
    elif bearer_token:
        headers["Authorization"] = f"Bearer {bearer_token}"
    else:
        raise ValueError(
            "You must provide either an api_key or a bearer_token.")

    response = requests.get(url, params=params, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(
            f"Request failed: {response.status_code} - {response.text}")

## Examples

In [9]:
bbox = "-122.301659, 47.443546, -122.130821, 47.643543"  # Format is <minLon,minLat,maxLon,maxLat>

data = get_traffic_incidents(bbox, subscription_key)

print(json.dumps(data, indent=4))

{
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "id": 12528037836014000,
            "geometry": {
                "type": "Point",
                "coordinates": [
                    -122.2890839684,
                    47.4815784556
                ]
            },
            "properties": {
                "incidentType": "Congestion",
                "startTime": "2025-09-01T11:53:30Z",
                "endTime": "2025-09-01T12:32:00Z",
                "lastModifiedTime": null,
                "description": "Backed up traffic 34th Ave S",
                "severity": 2,
                "isRoadClosed": false,
                "title": "34th Ave S",
                "isTrafficJam": true,
                "delay": 91,
                "endPoint": {
                    "type": "Point",
                    "coordinates": [
                        -122.2918184805,
                        47.4816133503
                    ]
         

In [10]:
for feature in data.get("features", []):
    props = feature["properties"]
    coords = feature["geometry"]["coordinates"]
    print(f"🛑 {props['incidentType']} | {props['title']}")
    print(f"   📍 Location: {coords[1]:.4f}, {coords[0]:.4f}")
    print(f"   📝 {props['description']}")
    print(f"   ⏰ {props['startTime']} → {props.get('endTime')}")
    print(f"   🚦 Severity: {props['severity']}, Delay: {props['delay']} sec")
    print()

🛑 Congestion | 34th Ave S
   📍 Location: 47.4816, -122.2891
   📝 Backed up traffic 34th Ave S
   ⏰ 2025-09-01T11:53:30Z → 2025-09-01T12:32:00Z
   🚦 Severity: 2, Delay: 91 sec

🛑 Construction | NE 44th St
   📍 Location: 47.5324, -122.1960
   📝 Roadworks NE 44th St
   ⏰ 2025-07-21T14:49:00Z → None
   🚦 Severity: 0, Delay: 0 sec

🛑 Construction | 112th Ave/SE 64th St (Lake Washington Ramp)
   📍 Location: 47.5338, -122.1958
   📝 Roadworks 112th Ave/SE 64th St (Lake Washington Ramp)
   ⏰ 2025-07-21T12:22:30Z → None
   🚦 Severity: 0, Delay: 0 sec

🛑 Construction | SE Maple Valley Hwy
   📍 Location: 47.4836, -122.1954
   📝 Roadworks SE Maple Valley Hwy
   ⏰ 2025-08-13T11:30:00Z → None
   🚦 Severity: 0, Delay: 0 sec

🛑 Construction | 44th St/Exit 7 (I-405)
   📍 Location: 47.5535, -122.1925
   📝 Roadworks 44th St/Exit 7 (I-405)
   ⏰ 2025-07-04T16:05:30Z → None
   🚦 Severity: 0, Delay: 0 sec

🛑 Construction | Northup Way (120th Ave NE)
   📍 Location: 47.6268, -122.1804
   📝 Roadworks Northup Way

In [11]:
records = []

for feature in data.get("features", []):
    props = feature["properties"]
    coords = feature["geometry"]["coordinates"]
    records.append({
        "Type": props["incidentType"],
        "Title": props["title"],
        "Description": props["description"],
        "Latitude": coords[1],
        "Longitude": coords[0],
        "Severity": props["severity"],
        "Delay (s)": props["delay"],
        "Start": props["startTime"],
        "End": props["endTime"]
    })

df = pd.DataFrame(records)
df

,Type,Title,Description,Latitude,Longitude,Severity,Delay (s),Start,End
0,Congestion,34th Ave S,Backed up traffic 34th Ave S,47.481578,-122.289084,2,91,2025-09-01T11:53:30Z,2025-09-01T12:32:00Z
1,Construction,NE 44th St,Roadworks NE 44th St,47.532370,-122.195986,0,0,2025-07-21T14:49:00Z,None
2,Construction,112th Ave/SE 64th St (Lake Washington Ramp),Roadworks 112th Ave/SE 64th St (Lake Washingto...,47.533758,-122.195786,0,0,2025-07-21T12:22:30Z,None
3,Construction,SE Maple Valley Hwy,Roadworks SE Maple Valley Hwy,47.483597,-122.195362,0,0,2025-08-13T11:30:00Z,None
4,Construction,44th St/Exit 7 (I-405),Roadworks 44th St/Exit 7 (I-405),47.553506,-122.192508,0,0,2025-07-04T16:05:30Z,None
5,Construction,Northup Way (120th Ave NE),Roadworks Northup Way (120th Ave NE),47.626757,-122.180433,0,0,2025-07-29T20:29:18Z,None


In [12]:
m = folium.Map(location=[47.50, -122.20], zoom_start=12)

for feature in data["features"]:
    props = feature["properties"]
    coords = feature["geometry"]["coordinates"]
    folium.Marker(
        location=[coords[1], coords[0]],
        popup=
        f"{props['title']}<br>{props['description']}<br>Severity: {props['severity']}",
        icon=folium.Icon(
            color="red" if props["severity"] > 2 else "orange")).add_to(m)

output_file = os.path.join(RESULTS_DIR, "traffic_map.html")
m.save(output_file)
print(f"✅ Map saved as {output_file}")

✅ Map saved as results/traffic_map.html


In [13]:
IFrame(src=output_file, width=1024, height=860)

In [14]:
map_link = FileLink(path=output_file)
map_link

/mnt/batch/tasks/shared/LS_root/mounts/clusters/seretkow8/code/Users/seretkow/azure maps/results/traffic_map.html

In [15]:
m = folium.Map(
    location=[47.50, -122.20],
    zoom_start=12,
    tiles=
    "https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
    attr="Esri World Imagery")

# Add markers for incidents
for feature in data["features"]:
    props = feature["properties"]
    coords = feature["geometry"]["coordinates"]

    folium.Marker(
        location=[coords[1], coords[0]],
        popup=
        f"<b>{props['title']}</b><br>{props['description']}<br>Severity: {props['severity']}",
        icon=folium.Icon(
            color="red" if props["severity"] > 2 else "orange")).add_to(m)

output_file = os.path.join(RESULTS_DIR, "traffic_map2.html")
m.save(output_file)
print(f"✅ Map saved as {output_file}")

✅ Map saved as results/traffic_map2.html


In [16]:
IFrame(src=output_file, width=1024, height=860)

In [17]:
map_link = FileLink(path=output_file)
map_link

/mnt/batch/tasks/shared/LS_root/mounts/clusters/seretkow8/code/Users/seretkow/azure maps/results/traffic_map2.html

In [18]:
m = folium.Map(
    location=[47.50, -122.20],
    zoom_start=12,
    tiles=
    "https://server.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer/tile/{z}/{y}/{x}",
    attr="Tiles © Esri — Source: Esri, USGS, NOAA")

# Add markers for incidents
for feature in data["features"]:
    props = feature["properties"]
    coords = feature["geometry"]["coordinates"]

    folium.Marker(
        location=[coords[1], coords[0]],
        popup=
        f"<b>{props['title']}</b><br>{props['description']}<br>Severity: {props['severity']}",
        icon=folium.Icon(
            color="red" if props["severity"] > 2 else "orange")).add_to(m)

output_file = os.path.join(RESULTS_DIR, "traffic_map3.html")
m.save(output_file)
print(f"✅ Map saved as {output_file}")

✅ Map saved as results/traffic_map3.html


In [19]:
IFrame(src=output_file, width=1024, height=860)

In [20]:
map_link = FileLink(path=output_file)
map_link

/mnt/batch/tasks/shared/LS_root/mounts/clusters/seretkow8/code/Users/seretkow/azure maps/results/traffic_map3.html

In [21]:
m = folium.Map(
    location=[47.50, -122.20],
    zoom_start=12,
    tiles=
    "https://server.arcgisonline.com/ArcGIS/rest/services/World_Street_Map/MapServer/tile/{z}/{y}/{x}",
    attr="Tiles © Esri — Source: Esri, USGS, NOAA")

# Add markers for incidents
for feature in data["features"]:
    props = feature["properties"]
    coords = feature["geometry"]["coordinates"]

    folium.Marker(
        location=[coords[1], coords[0]],
        popup=
        f"<b>{props['title']}</b><br>{props['description']}<br>Severity: {props['severity']}",
        icon=folium.Icon(
            color="red" if props["severity"] > 2 else "orange")).add_to(m)

output_file = os.path.join(RESULTS_DIR, "traffic_map4.html")
m.save(output_file)
print(f"✅ Map saved as {output_file}")

✅ Map saved as results/traffic_map4.html


In [22]:
IFrame(src=output_file, width=1024, height=860)

In [23]:
map_link = FileLink(path=output_file)
map_link

/mnt/batch/tasks/shared/LS_root/mounts/clusters/seretkow8/code/Users/seretkow/azure maps/results/traffic_map4.html